# 装饰器的使用及Pyinstaller相关问题
## 一、装饰器的使用：

In [1]:
#定义要被签名的函数：
def sumi(args):
    s = '' if type(list(args)[0]) == str else 0
    for i in list(args):
        s += i
    return s

In [2]:
#管理员和用户的数据库
admin_info = {'lizhoubo': 'ilovexh'}
users_info = {'lizhoubo': 'ilovexh'}


#管理员操作系统：管理装饰器使用
class apply_user(object):
    def __init__(self):
        self.id = input('欢迎您登陆管理员系统！\n请输入管理员账号：\n')
        if not self.id in admin_info:
            raise Warning('您不是系统管理员，请申请管理权限！')
        else:
            self.password = input('请输入您的密码：\n')
            if admin_info[self.id] != self.password:
                raise Warning('密码错误，认证失败！')
            else:
                case = eval(
                    input('恭喜您，登陆成功！\n你可以使用以下函数进行操作：\n ' +
                          '1、添加新用户；\n 2、删除用户；\n 3、修改密码；\n 4、查询注册表；\n 5、退出。\n'))
        cases = [self.add_user, self.del_user, self.alter_pass, self.info_get]
        while True:
            if case in range(1, 5):
                print(cases[case - 1]())
            elif case == 5:
                print('成功退出操作系统！但您可自行调用有关函数。')
                break
            else:
                raise Warning('无效操作！')
            case = eval(
                input('你可以使用以下函数进行操作：\n ' +
                      '1、添加新用户；\n 2、删除用户；\n 3、修改密码；\n 4、查询注册表；\n 5、退出。\n'))

    def add_user(self):
        id_ = input('请输入要注册的账号：\n')
        password = input('请输入密码：\n')
        if id_ in users_info:
            raise Warning('该账户已被注册！请直接登陆！')
        users_info[id_] = password
        return '注册成功！可以登陆用户。'

    def del_user(self):
        id_ = input('请输入要注销的账号：\n')
        if not id_ in users_info:
            raise Warning('该账户不存在，或请勿重复操作！')
        users_info.pop(id_)
        return '删除成功！'

    def alter_pass(self):
        password = input('修改您的密码为：\n')
        admin_info[self.id] = password
        users_info[self.id] = password
        return '密码修改成功！'

    def info_get(self):
        info = users_info
        return info

In [3]:
#函数签名数据库
rpc_info = {}


#装饰器实例：XML-RPC协议提供签名
class func_use(object):
    def __init__(self):
        self.id = input('请输入您的账号：\n')
        if not self.id in users_info:
            raise Warning('账号不存在，请向黎洲波申请账号！')
        else:
            self.password = input('请输入您的密码：\n')
            if users_info[self.id] != self.password:
                raise Warning('密码错误，登陆失败！')
            else:
                print('恭喜您，登录成功！您可以使用{}函数。'.format('xmlrpc'))

    def xmlrpc(self, in_=(), out=(type(None), )):
        def _xmlrpc(function):
            #注册签名
            func_name = function.__name__
            rpc_info[func_name] = (in_, out)

            def _check_type(elements, types):
                """Subfunction that checks the types."""
                if len(elements) != len(types):
                    raise TypeError('arguement count is wrong')

                typed = enumerate(zip(elements, types))

                for index, couple in typed:
                    arg, of_the_right_type = couple

                    if isinstance(arg, of_the_right_type):
                        continue
                    raise TypeError('arg #%d should be %s' %
                                    (index, of_the_right_type))

            #封装函数
            def __xmlrpc():  #没有允许的关键字
                s = 'int' if in_[0] is int else 'str'
                args = eval(
                    input('请输入%d个{}型数据,以英文逗号分开：\n'.format(s) % len(in_)))

                #检查输出内容
                checkable_args = list(args)
                _check_type(checkable_args, in_)

                #执行函数
                res = function(args)

                #检查输出
                if not type(res) in (tuple, list):
                    checkable_res = (res, )
                else:
                    checkable_res = res
                _check_type(checkable_res, out)

                #检查完毕，返回函数
                print('这些玩意的和是：{}'.format(res))
                return res

            return __xmlrpc

        return _xmlrpc

<br>运行流程类似下列操作（这里便于后面pyinstaller打包运行）：

In [4]:
if __name__ == '__main__':
    try:
        a = apply_user()
        b = func_use()
        b.xmlrpc([int] * 5, [int])(sumi)()
    except Warning:
        input('Warning:您遇到了错误！\n请按回车键退出。')
    else:
        input('运行完毕，请按回车键结束程序。')    

欢迎您登陆管理员系统！
请输入管理员账号：
lizhoubo
请输入您的密码：
ilovexh
恭喜您，登陆成功！
你可以使用以下函数进行操作：
 1、添加新用户；
 2、删除用户；
 3、修改密码；
 4、查询注册表；
 5、退出。
4
{'lizhoubo': 'ilovexh'}
你可以使用以下函数进行操作：
 1、添加新用户；
 2、删除用户；
 3、修改密码；
 4、查询注册表；
 5、退出。
1
请输入要注册的账号：
xiaohei
请输入密码：
ilovelzb
注册成功！可以登陆用户。
你可以使用以下函数进行操作：
 1、添加新用户；
 2、删除用户；
 3、修改密码；
 4、查询注册表；
 5、退出。
4
{'lizhoubo': 'ilovexh', 'xiaohei': 'ilovelzb'}
你可以使用以下函数进行操作：
 1、添加新用户；
 2、删除用户；
 3、修改密码；
 4、查询注册表；
 5、退出。
5
成功退出操作系统！但您可自行调用有关函数。
请输入您的账号：
xiaohei
请输入您的密码：
ilovelzb
恭喜您，登录成功！您可以使用xmlrpc函数。
请输入5个int型数据,以英文逗号分开：
1,2,3,5,8
这些玩意的和是：19
运行完毕，请按回车键结束程序。


被签名的函数和特定参数输出到字典rpc_info里：

In [5]:
print(rpc_info)

{'sumi': ([<class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>, <class 'int'>], [<class 'int'>])}


## 二、用pyinstaller打包.py文件
### 0、这里涉及到问题：
- [安装和使用Pyinstaller](#安装)
<br><br>
- [递归深度达上限](#递归问题)
<br><br>
- [关联文件无法加载](#文件)
<br><br>
- [Anaconda和Pipenv环境下打包](#Ana)

### 1、安装和使用Pyinstaller <a name ='安装'></a>

- **安装**

Windows系统在命令行用`conda install pyinstaller`或`pip install pyinstaller`安装Pyinstaller包。
<br><br>
升级Pyinstaller用`pip --upgrade pyinstaller`。

- **使用**

在要打包的.py文件目录下命令行`pyinstaller -F xx.py`。
<br><br>
这里的`-F`是Pyinstaller的操作：打包为一个.exe文件，且用DOS窗口运行。
<br><br>
Pyinstaller命令可用`pyinstaller help`查看。

- **详见[官方文档](https://pythonhosted.org/PyInstaller/usage.html)**

### 2、递归深度达上限 <a name ='递归问题'></a>

- **显示`Error:maximum recursion depth exceeded.`**

Python调用堆栈的上限递归深度大概只有近1000（一般情况下是够用的），这里使用Sys包进行设置。
<br><br>
Pyinstaller打包失败后虽然不能生成.exe，但是生成了.spec文件，用记事本打开.spec文件，在文件第二行加上：

In [6]:
import sys
sys.setrecursionlimit(100000) #括号里为设置的递归深度

然后命令行运行`pyinstaller -F xx.spec`就可以了。

### 3、关联文件无法加载<a name ='文件'></a>

- **运行文件时显示`Failed to execute script xx`**

.py文件运行依赖电脑指定路径的关联文件时，pyinstaller打包成.exe时无法找到路径。
<br><br>
可以通过修改.spec文件解决问题，这里没操作过，不赘述，详见:
<br>
- https://pythonhosted.org/PyInstaller/spec-files.html
- https://www.yuanrenxue.com/tricks/pyinstaller-spec.html

### 4、Anaconda和Pipenv环境下打包<a name ='Ana'></a>

- **打包出来包含可执行程序的文件很大**

在Anaconda的运行环境下Pyinstaller打包时会将Anaconda环境中各种包也打包到文件中，导致了打包速度慢且打包出来过大，不好传播的问题。
<br><br>
当然也可以自己在运行Pyinstaller时命令`hide-import-xxx`，但是这种方法要一个一个包hide，不太好用。
<br><br>
我也想到直接用女朋友的原生`Python 3.6`环境打包，这样文件也不会很大。<mark>*但是这里还有更好的方法：*

- **Pipenv环境**

Windows系统命令行输入`pip install pipenv`安装Pipenv包。
<br><br>
在xxx.py所在目录运行DOS命令行输入`pipenv shell`进入.virtualenv。
<br><br>
这里查看自己是否操作正确可以用命令`pip list`查看当前虚拟环境下安装的包，也可以用`pipenv --venv` 查看虚拟环境路径。更多操作见`pipenv help`
<br><br>
然后就可以使用Pyinstaller了，<mark>操作跟前面一样</mark>。
<br><br>
打包完成可以用`exit`命令退出虚拟环境。
<br><br>
Pipenv使用详见[官方文档](https://docs.pipenv.org/en/latest/)